In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("gabrielsantello/cars-purchase-decision-dataset")

print("Path to dataset files:", path)

In [ ]:
import pandas as pd
data= pd.read_csv('path to dataset')
data

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

X=np.array([data['Age'],data['AnnualSalary']])
Y=np.array(data['Purchased'])
X=X.T
X.shape

X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2)

In [ ]:
from collections import Counter

#Task 2
def cross_entropy(Y):
    count=Counter(Y)
    P=[]
    for i in count.values():
        P.append(i/len(Y))
    P=np.array(P)
    return -np.sum(np.dot(P,np.log2(P)))


#Task 1

print(cross_entropy(Y_train))


In [ ]:
#Task 3

def split(X,Y):

    for j in range(X.shape[1]):
        thresholds=np.unique(X[:,j]) #get all unique values as thresholds

    min_entropy=float('inf')
    left_split,right_split=[],[]
    left_split_l,right_split_l=[],[]
    best_feature,best_threshold=0,0

    for j in range(X.shape[1]):
        
            
        for threshold in thresholds:
            
            Left,Right=[],[]
            Left_l,Right_l=[],[]

            for i in range(X.shape[0]):
                    if X[i,j]<= threshold:
                        Left.append(X[i])
                        Left_l.append(Y[i])
                    else:
                        Right.append(X[i])
                        Right_l.append(Y[i])
        
            left_w=len(Left)/Y.shape[0]
            right_w=len(Right)/Y.shape[0]

            entropy=left_w*cross_entropy(np.array(Left_l))+right_w*cross_entropy(np.array(Right_l))

            if entropy<min_entropy:
                min_entropy=entropy
                left_split,right_split=Left,Right
                left_split_l,right_split_l=Left_l,Right_l
                best_feature=j
                best_threshold=threshold
            

    return left_split,right_split,left_split_l,right_split_l,min_entropy,best_feature,best_threshold

left,right,left_l,right_l,entropy,feature,best_threshold=split(X_train,Y_train)
print(f"min-weighed entropy= {entropy}")
print(f"best feature= {feature}")
print(f"best threshold= {best_threshold}")
        

In [ ]:
#Task 4

class DecisionTree:
    def __init__(self,threshold=None,label=None,feature=None):

        # node values to store
        self.feature=feature
        self.threshold=threshold
        self.label=label # label for leaf node for prediction

        #pointers
        self.left=None
        self.right=None


def build_tree(depth,n,X,Y):
    if Y.shape[0]==1 :
        return Y[0]
    
    if all(Y==Y[0]):
        return DecisionTree(label=Y[0]) # check if all labels are same
    
    if depth==n:
        return DecisionTree(label=np.bincount(Y).argmax()) #returning the most common label
    
    left,right,left_l,right_l,entropy,feature,threshold=split(X,Y)

    node=DecisionTree(feature=feature,threshold=threshold)

    node.left=build_tree(depth+1,n,np.array(left),np.array(left_l))
    node.right=build_tree(depth+1,n,np.array(right),np.array(right_l))

    return node

def train(X,Y,max_depth):
    return build_tree(0,max_depth,X,Y)

Tree=train(X_train,Y_train,5)


In [ ]:

# Task 5
def predict(X,Node):
    if Node.left==None or Node.right==None:
        return Node.label
    
    if X[Node.feature]<=Node.threshold:
        return predict(X,Node.left)
    else:
        return predict(X,Node.right)
    

result=0
def prediction(X,Tree):
    for sample in X:
        result=predict(sample,Tree)
        print(f"class predicted for sample {sample} is {result}")

prediction(X_test,Tree)


In [ ]:
accuracy=np.mean(Y_test==result)
print(f'accuracy for the current dataset = {accuracy}')

In [ ]:
#Task 6

import networkx as nx
import matplotlib.pyplot as plt

def add_edges(nx_tree,node,parent=None,edge_label=""):
    if node is None:
        return
    
    if node.label is not None:
        node_label = f"Label: {node.label}"
    else:
        node_label = f"{node.feature} <= {node.threshold}"

    nx_tree.add_node(id(node), label=node_label)

    if parent is not None:
        nx_tree.add_edge(parent, id(node), label=edge_label)

    add_edges(nx_tree, node.left, id(node), "Yes")
    add_edges(nx_tree, node.right, id(node), "No")
    
def draw_tree(Tree):
    nx_tree=nx.DiGraph()
    add_edges(nx_tree,Tree)
    pos = nx.spring_layout(nx_tree)  
    labels = nx.get_node_attributes(nx_tree, "label")
    edge_labels = nx.get_edge_attributes(nx_tree, "label")

    plt.figure(figsize=(10, 6))
    nx.draw(nx_tree, pos, with_labels=True, labels=labels, node_size=3000, node_color="lightblue", edge_color="black")
    nx.draw_networkx_edge_labels(nx_tree, pos, edge_labels=edge_labels)
    
    plt.show()

draw_tree(Tree)
